<a href="https://colab.research.google.com/github/Felipe34515/Trabajos-BI/blob/main/LAB1_BI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LABORATORIO 1 - INTELIGENCIA DE NEGOCIOS**

Santiago Pardo - 202013025



Luis Plazas - 202013155




Felipe Rueda - 202010903

## DESCRIPCIÓN DEL PROBLEMA

### Contexto



En el vasto panorama del mundo empresarial, la comprensión profunda de los clientes es la clave para el éxito sostenible. La segmentación de clientes emerge como una poderosa herramienta estratégica que redefine la forma en que las empresas abordan sus mercados. Este proceso implica la división cuidadosa y deliberada de la base de clientes en grupos homogéneos, con características y comportamientos similares. La segmentación de clientes va más allá de la simple categorización; es la llave maestra que desbloquea el potencial de la personalización. Al centrarse en las similitudes y preferencias compartidas dentro de cada segmento, las empresas pueden anticipar las necesidades de sus clientes, crear mensajes más efectivos y diseñar productos y servicios que resuenen con precisión.



AlpesInsight, es una empresa pionera en la intersección de servicios financieros y tecnología. En su búsqueda constante para ofrecer experiencias personalizadas y seguras a sus clientes, AlpesInsight está expandiendo su equipo con la incorporación de expertos en aprendizaje automático para enfrentar el desafío de aprovechar datos de transacciones con tarjetas de crédito para comprender los patrones de comportamiento y preferencias de los clientes. AlpesInsight se esfuerza por desarrollar soluciones innovadoras que permitan personalizar ofertas financieras y servicios, mejorando así la satisfacción del cliente y fortaleciendo la seguridad de las transacciones. El equipo ha dedicido utilizar técnicas de agrupación para resolver este reto



### Diccionario de los Datos (En Inglés)



| Atributo                         | Significado                                                                                                                   |
| -------------------------------- | ----------------------------------------------------------------------------------------------------------------------------- |
| CUST_ID                          | Credit card holder ID                                                                                                         |
| BALANCE                          | Monthly average balance (based on daily balance averages)                                                                     |
| BALANCE_FREQUENCY                | Ratio of last 12 months with balance. (1: Frequently updated, 0: Not frequently updated)                                      |
| PURCHASES                        | Total purchase amount spent during last 12 months                                                                             |
| ONEOFF_PURCHASES                 | Total amount of one-off purchases                                                                                             |
| INSTALLMENTS_PURCHASES           | Total amount of installment purchases                                                                                         |
| CASH_ADVANCE                     | Total cash-advance amount                                                                                                     |
| PURCHASES_FREQUENCY              | Frequency of purchases (Percent of months with at least one purchase). (1: Frequently purchased, 0: Not frequently purchased) |
| ONEOFF_PURCHASES_FREQUENCY       | Frequency of one-off-purchases. (1: Frequently purchased, 0: Not frequently purchased)                                        |
| PURCHASES_INSTALLMENTS_FREQUENCY | Frequency of installment purchases. (1: Frequently purchased, 0: Not frequently purchased)                                    |
| CASHADVANCE_FREQUENCY            | Cash-Advance frequency                                                                                                        |
| CASH_ADVANCE_TRX                 | Average amount per cash-advance transaction                                                                                   |
| PURCHASES_TRX                    | Average amount per purchase transaction                                                                                       |
| CREDIT_LIMIT                     | Credit limit                                                                                                                  |
| PAYMENTS                         | Total payments (Due amount paid by the customer to decrease their statement balance) in the period                            |
| MINIMUM_PAYMENTS                 | Total minimum payments due in the period                                                                                      |
| PRC_FULL_PAYMEN                  | Percentage of months with full payment of the due statement balance                                                           |
| TENURE                           | Number of months as a customer                                                                                                |

## ENTENDIMIENTO DE LOS DATOS

(Aquí pegas el tablero de control)

## PREPARACIÓN DE DATOS

## MODELAMIENTO

## VALIDACIÓN